## Import

In [2]:
import pickle
import os
import pandas as pd
import matplotlib.pyplot as plt
import re
import geopandas as gpd
from PIL import Image, ImageDraw
import io

### Open the pickle

In [3]:
pickle_file = '../assets/pickles/df_communes.pkl'
if os.path.exists(pickle_file):
	with open(pickle_file, 'rb') as file:
		dataframes = pickle.load(file)

### National analysis

In [4]:
regex='^voix(?!.*ratio)(?!.*T2)'
votes_per_candidates = dataframes['2022'].filter(regex=regex).columns.tolist()
columns_kept = ['dep', 'nomdep', 'exprimes'] + votes_per_candidates
columns_kept

['dep',
 'nomdep',
 'exprimes',
 'voixARTHAUD',
 'voixPOUTOU',
 'voixROUSSEL',
 'voixMELENCHON',
 'voixJADOT',
 'voixHIDALGO',
 'voixLASSALLE',
 'voixMACRON',
 'voixPECRESSE',
 'voixZEMMOUR',
 'voixDUPONTAIGNAN',
 'voixMLEPEN']

Consolidate data around:
- First round
- Per departement
- Votes on the political spectrum

In [5]:
dataframes['2022'].loc[(dataframes['2022']['dep'] == '01') & (dataframes['2022']['exprimes'] %  1 != 0), 'exprimes']

7        227.999992
48     15991.999512
116      274.000008
235       90.000004
278      133.000004
306      853.000031
374      866.999969
Name: exprimes, dtype: float64

`Exprimes` is set to float, changing the data type to int and making sure it's correctly rounded.

In [6]:
dataframes['2022']['exprimes'] = dataframes['2022']['exprimes'].round(0).astype(int)


In [7]:
df_departement = {}
regex='^voix(?!.*ratio)(?!.*T2)'
for year, df in dataframes.items():
	# Moving to list might be a poor idea performance wise
	votes_per_candidates = dataframes[year].filter(regex=regex).columns.tolist()
	columns_kept = ['dep', 'nomdep', 'exprimes'] + votes_per_candidates
	df_departement[year] = df[columns_kept].groupby(['dep', 'nomdep']).sum().reset_index()
	

In [8]:
pd.set_option('display.max_rows', 100)
regex_votes = r'^voix'
sum_votes = df_departement['2022'].filter(regex=regex_votes).sum(axis=1)
check_votes = (df_departement['2022'].filter(regex=regex_votes).sum(axis=1) == df_departement['2022']['exprimes'])
check_votes

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
20    True
21    True
22    True
23    True
24    True
25    True
26    True
27    True
28    True
29    True
30    True
31    True
32    True
33    True
34    True
35    True
36    True
37    True
38    True
39    True
40    True
41    True
42    True
43    True
44    True
45    True
46    True
47    True
48    True
49    True
50    True
51    True
52    True
53    True
54    True
55    True
56    True
57    True
58    True
59    True
60    True
61    True
62    True
63    True
64    True
65    True
66    True
67    True
68    True
69    True
70    True
71    True
72    True
73    True
74    True
75    True
76    True
77    True
78    True
79    True
80    True
81    True
82    True
83    True
84    True
85    True
86    True
87    True
88    True
89    True
90    True

In [9]:
df_departement['2002']

,dep,nomdep,exprimes,voixLAGUILLER,voixBESANCENOT,voixGLUCKSTEIN,voixHUE,voixTAUBIRA,voixMAMERE,voixCHEVENEMENT,voixJOSPIN,voixLEPAGE,voixSAINTJOSSE,voixBAYROU,voixCHIRAC,voixMADELIN,voixBOUTIN,voixMEGRET,voixLEPEN
0,01,AIN,2.406520e+05,11562,9404,1077,5134,4527,12339,14665,30418,5258,9718,18614,41348,12580,2966,8425,52617
1,02,AISNE,2.618030e+05,21369,12412,1877,10286,3228,9379,12230,40209,3727,10646,13972,49280,8462,2333,6831,55562
2,03,ALLIER,1.797680e+05,11241,8467,973,14616,2669,6961,9337,28814,2597,7936,11576,37533,6018,2375,3268,25387
3,04,ALPES DE HAUTE PROVENCE,7.943000e+04,4366,4056,421,3465,1700,4468,4654,10845,1459,7123,4651,12998,2943,845,2252,13184
4,05,HAUTES ALPES,6.940200e+04,3437,3797,320,2376,1664,4677,4068,9152,1593,5063,4816,12036,3579,926,1933,9965
5,06,ALPES MARITIMES,4.670400e+05,17544,11992,1337,13984,6939,22388,22634,56868,8283,11430,31729,102588,22671,3466,11825,121362
6,07,ARDECHE,1.614450e+05,8613,8287,870,6717,3086,8467,9072,22946,3039,12391,10984,27861,5289,2536,4244,27043
7,08,ARDENNES,1.344980e+05,9799,5863,664,4548,1490,5207,5637,21767,1817,5283,7551,24038,3914,1246,4848,30826
8,09,ARIEGE,7.960600e+04,4388,4442,349,4162,1476,4207,3703,18890,1076,6684,3236,11394,1629,687,1256,12027
9,10,AUBE,1.381400e+05,7731,4990,734,4417,2071,5072,6485,18882,2490,6906,8608,28915,5661,1476,3693,30009


### Save the consolidated data at departement level

In [10]:
with open("../assets/pickles/df_departements.pkl", "wb") as file:
    pickle.dump(df_departement, file)